In [1]:
import parselmouth
from parselmouth.praat import call
import tensorflow as tf
import numpy as np

In [2]:
def measurePitch(voiceID, f0min, f0max, unit):
    sound = parselmouth.Sound(voiceID) # read the sound
    pitch = call(sound, "To Pitch", 0.0, f0min, f0max) #create a praat pitch object
    meanF0 = call(pitch, "Get mean", 0, 0, unit) # get mean pitch
    harmonicity = call(sound, "To Harmonicity (cc)", 0.01, 75, 0.1, 1.0)
    hnr = call(harmonicity, "Get mean", 0, 0)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    localJitter = call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
    localabsoluteJitter = call(pointProcess, "Get jitter (local, absolute)", 0, 0, 0.0001, 0.02, 1.3)
    rapJitter = call(pointProcess, "Get jitter (rap)", 0, 0, 0.0001, 0.02, 1.3)
    ppq5Jitter = call(pointProcess, "Get jitter (ppq5)", 0, 0, 0.0001, 0.02, 1.3)
    ddpJitter = call(pointProcess, "Get jitter (ddp)", 0, 0, 0.0001, 0.02, 1.3)
    localShimmer =  call([sound, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    localdbShimmer = call([sound, pointProcess], "Get shimmer (local_dB)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq3Shimmer = call([sound, pointProcess], "Get shimmer (apq3)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    aqpq5Shimmer = call([sound, pointProcess], "Get shimmer (apq5)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq11Shimmer =  call([sound, pointProcess], "Get shimmer (apq11)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    ddaShimmer = call([sound, pointProcess], "Get shimmer (dda)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    

    return meanF0, rapJitter , localabsoluteJitter, ppq5Jitter, ddpJitter, localdbShimmer, apq3Shimmer, aqpq5Shimmer, ddaShimmer, hnr

In [3]:
wave_file = r'INPUTANYWAVFILEHERE'
sound = parselmouth.Sound(wave_file)
(meanF0, rapJitter, localabsoluteJitter, ppq5Jitter, ddpJitter, localdbShimmer, apq3Shimmer, aqpq5Shimmer, ddaShimmer, hnr) = measurePitch(sound, 75, 500, "Hertz")


In [4]:
test_sound_m = [meanF0, rapJitter, localabsoluteJitter, ppq5Jitter, ddpJitter, localdbShimmer, apq3Shimmer, aqpq5Shimmer, ddaShimmer, hnr]

In [5]:
model = tf.keras.models.load_model('VocalModel.h5')


In [7]:
model.predict(np.array(test_sound_m).reshape(1,-1))

1/1 [==============================] - 0s 107ms/step


array([[0.42670012]], dtype=float32)